In [ ]:
import pandas as pd
import pyspark
sc = pyspark.SparkContext('local[*]')

In [ ]:
postulantes_edu = pd.read_csv('Sources/fiuba_1_postulantes_educacion_concat.csv')
postulantes_gen = pd.read_csv('Sources/fiuba_2_postulantes_genero_y_edad_concat.csv')

In [ ]:
postulantes_edu.drop(['Unnamed: 0'],axis=1,inplace=True)
postulantes_edu.rename(columns={'nombre':'estudio'},inplace=True)
postulantes_edu.head()

In [ ]:
postulantes_gen.drop(['Unnamed: 0'],axis=1,inplace=True)
postulantes_gen.head()

In [ ]:
estudios = pd.get_dummies(postulantes_edu['estudio'])
for col in estudios.columns:
    estudios.loc[estudios.loc[:,col]==1,col] = postulantes_edu.loc[estudios.loc[:,col]==1,'estado']

In [ ]:
estudios.head()

In [ ]:
postulantes_edu = postulantes_edu.merge(estudios, left_index=True, right_index=True)
postulantes_edu.drop(['estudio','estado'], axis=1, inplace=True)
postulantes_edu.head()

In [ ]:
estados = ['','Abandonado', 'En Curso', 'Graduado']
for x in range(len(estados)):
    for col in postulantes_edu.columns:
        postulantes_edu.loc[postulantes_edu.loc[:,col]==estados[x],col] = x

In [ ]:
groups = postulantes_edu.groupby('idpostulante')


In [ ]:
df = pd.DataFrame()


In [ ]:
import datetime
#df = pd.DataFrame()
df = []
i = 0
for indice in groups.indices:
    i = i + 1
    aux = groups.get_group(indice).max()
    aux = aux.tolist()
    df.append(aux)
df = pd.DataFrame.from_records(df)
df.head()


In [ ]:
for x in df.columns:
    nwName = postulantes_edu.columns[x]
    df.rename(columns={x:nwName}, inplace=True)

In [ ]:
df = df.merge(postulantes_gen,on='idpostulante',how='outer')


In [ ]:
df.to_csv('Sources/postulantes_proc.csv')


In [ ]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
postulantes.head()

In [ ]:
def max_educacion(row):
    lista_de_niveles = ['Doctorado','Master','Posgrado','Universitario','Otro','Terciario/Técnico','Secundario']
    lista_de_indices = ['Secundario','Terciario/Técnico','Otro','Universitario','Posgrado','Master','Doctorado']
    lista_de_estados = [0,2, 1, 3]
    for nivel in lista_de_niveles:
        if row[nivel]==0: continue
        for estado in lista_de_estados:
            if row[nivel] == estado:
                nivel_def = lista_de_indices.index(nivel)
                return ((3*nivel_def) + estado)
    return -1
        


In [ ]:
postulantes['nivel'] = postulantes.apply(lambda x: max_educacion(x), axis=1)

In [ ]:
postulantes.head()

In [ ]:
def dates_format(date):
    try:
        date_new = pd.to_datetime('today').year-pd.to_datetime(date).year
        #date_new = int(date_new)
        return date_new
    except ValueError:
        return -1

In [ ]:
fechasRDD = sc.parallelize(postulantes['fechanacimiento'])
postulantes['fechanacimiento'] = fechasRDD.map(lambda x: dates_format(x)).collect()
fechasRDD = None

In [ ]:
postulantes.drop(['Doctorado','Master','Posgrado','Universitario','Otro','Terciario/Técnico','Secundario'], axis=1, inplace=True)
postulantes.head()

In [ ]:
postulantes.to_csv('Sources/postulantes_proc.csv')